In [1]:
import os
import random
import math
import asyncio
import nest_asyncio
from spade.agent import Agent
from spade.behaviour import CyclicBehaviour
from spade.template import Template

# Aplicar parche para Jupyter
nest_asyncio.apply()

# =============================================================================
# 1. GENERAR HTML (GUI)
# =============================================================================
os.makedirs("static", exist_ok=True)

html_content = """
<!DOCTYPE html>
<html>
<head>
    <title>PC3: Seleccion Natural</title>
    <style>
        body { font-family: sans-serif; background: #222; color: white; text-align: center; }
        canvas { background: #333; border: 2px solid #555; margin-top: 10px; }
        #panel { background: #444; padding: 10px; display: inline-block; border-radius: 8px; }
    </style>
</head>
<body>
    <h1>Simulacion PC3 (Seleccion Natural)</h1>
    <div id="panel">
        Día: <span id="dia">0</span> | Pob: <span id="pob">0</span> | 
        <label>Velocidad:</label> <input type="range" id="slider" min="1" max="10" value="1">
    </div>
    <br>
    <canvas id="miCanvas" width="800" height="600"></canvas>

    <script>
        const canvas = document.getElementById('miCanvas');
        const ctx = canvas.getContext('2d');
        const ANCHO = 800, ALTO = 600;

        function draw(data) {
            ctx.clearRect(0, 0, ANCHO, ALTO);

            // Zonas Seguras
            ctx.fillStyle = "rgba(100, 149, 237, 0.2)";
            ctx.fillRect(0, 0, 50, ALTO);
            ctx.fillRect(ANCHO - 50, 0, 50, ALTO);

            // Comida
            ctx.fillStyle = "#00ff00";
            data.comida.forEach(c => {
                ctx.beginPath(); ctx.arc(c.x, c.y, 4, 0, Math.PI*2); ctx.fill();
            });

            // Blobs
            data.blobs.forEach(b => {
                ctx.beginPath();
                // Color: Azul (lento) -> Rojo (rápido)
                let val = Math.min(1, (b.speed - 1.0) / 3.0); 
                let r = Math.floor(255 * val);
                let blue = Math.floor(255 * (1 - val));
                
                ctx.fillStyle = (b.state === 'seguro') ? '#555' : `rgb(${r}, 0, ${blue})`;
                ctx.arc(b.x, b.y, 8, 0, Math.PI*2);
                ctx.fill();
                ctx.strokeStyle = "white"; ctx.stroke();
            });

            document.getElementById('dia').innerText = data.dia;
            document.getElementById('pob').innerText = data.blobs.length;
        }

        async function loop() {
            try {
                // NOTA: Pedimos a la raíz /data, no relativo
                const res = await fetch('/data');
                if (res.ok) {
                    const data = await res.json();
                    draw(data);
                } else {
                    console.log("Error 404 o similar");
                }
            } catch(e) { console.log("Conectando..."); }
            setTimeout(loop, 100);
        }
        
        document.getElementById('slider').oninput = async function() {
            await fetch('/speed', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({val: this.value})
            });
        }
        loop();
    </script>
</body>
</html>
"""

with open("static/index.html", "w") as f:
    f.write(html_content)
    
print("✅ Archivo static/index.html actualizado.")


# =============================================================================
# 2. CLASES DEL MODELO
# =============================================================================
ANCHO = 800
ALTO = 600
MARGEN = 50
DURACION_DIA = 400

class Comida:
    def __init__(self):
        self.x = random.randint(MARGEN + 20, ANCHO - MARGEN - 20)
        self.y = random.randint(20, ALTO - 20)

class Blob:
    def __init__(self, speed=2.0, home_side=None):
        self.speed = speed
        self.energy = 0
        self.state = "hunting"
        self.home_side = home_side if home_side else random.choice(['left', 'right'])
        self.x = random.randint(0, MARGEN) if self.home_side == 'left' else random.randint(ANCHO - MARGEN, ANCHO)
        self.home_x = 0 if self.home_side == 'left' else ANCHO
        self.y = random.randint(0, ALTO)

    def update(self, food_list):
        if self.state == "seguro": return
        
        target_x, target_y = self.x, self.y
        if self.state == "hunting":
            closest, min_dist = None, 9999
            for f in food_list:
                d = math.hypot(f.x - self.x, f.y - self.y)
                if d < min_dist:
                    min_dist = d
                    closest = f
            if closest:
                target_x, target_y = closest.x, closest.y
            else:
                target_x = ANCHO / 2 + random.randint(-50, 50)
                target_y = self.y + random.randint(-50, 50)
        elif self.state == "returning":
            target_x, target_y = self.home_x, self.y

        dx = target_x - self.x
        dy = target_y - self.y
        dist = math.hypot(dx, dy)
        if dist > 0:
            self.x += (dx / dist) * self.speed
            self.y += (dy / dist) * self.speed
        
        self.x = max(0, min(ANCHO, self.x))
        self.y = max(0, min(ALTO, self.y))

        if self.state == "returning":
            if (self.home_side == 'left' and self.x < MARGEN) or \
               (self.home_side == 'right' and self.x > ANCHO - MARGEN):
                self.state = "seguro"

# =============================================================================
# 3. AGENTE MANAGER
# =============================================================================
class ManagerAgent(Agent):
    async def setup(self):
        print(f"Agente {self.jid} iniciando...")
        
        self.blobs = [Blob(speed=2.0) for _ in range(15)]
        self.comida = [Comida() for _ in range(30)]
        self.dia = 1
        self.timer = 0
        self.delay = 0.05

        # --- SERVIDOR WEB ---
        self.web.start(port=10000)
        
        # 1. Rutas de Datos (API)
        self.web.add_get("/data", self.data_controller, template=None)
        self.web.add_post("/speed", self.speed_controller, template=None)
        
        # 2. Rutas Estáticas (Al final y en carpeta /static para no tapar /data)
        self.web.app.router.add_static("/static", "static")
        
        print("🌐 GUI disponible en: http://localhost:10000/static/index.html")
        
        self.add_behaviour(self.GameLoopBehaviour())

    async def data_controller(self, request):
        return {
            "dia": self.dia,
            "blobs": [{"x": b.x, "y": b.y, "state": b.state, "speed": round(b.speed, 2)} for b in self.blobs],
            "comida": [{"x": c.x, "y": c.y} for c in self.comida]
        }

    async def speed_controller(self, request):
        data = await request.json()
        val = float(data.get("val", 1))
        self.delay = 0.1 / val
        return {"status": "ok"}

    class GameLoopBehaviour(CyclicBehaviour):
        async def run(self):
            agent = self.agent
            if agent.timer < DURACION_DIA:
                for b in agent.blobs:
                    b.update(agent.comida)
                    if b.state == "hunting":
                        for f in agent.comida[:]:
                            if math.hypot(b.x - f.x, b.y - f.y) < 10:
                                agent.comida.remove(f)
                                b.energy += 1
                                if b.energy >= 2: b.state = "returning"
                agent.timer += 1
            else:
                print(f"Fin Día {agent.dia}. Evolucionando...")
                survivors = []
                for b in agent.blobs:
                    is_safe = b.state == "seguro" or (b.home_side == 'left' and b.x < MARGEN) or (b.home_side == 'right' and b.x > ANCHO - MARGEN)
                    if is_safe and b.energy >= 1:
                        spawn_child = (b.energy >= 2)
                        b.x = 0 if b.home_side == 'left' else ANCHO
                        b.state, b.energy = "hunting", 0
                        survivors.append(b)
                        if spawn_child:
                            survivors.append(Blob(speed=max(0.5, b.speed + random.uniform(-0.5, 0.5)), home_side=b.home_side))
                
                agent.blobs = survivors
                agent.comida = [Comida() for _ in range(30)]
                agent.dia += 1
                agent.timer = 0
            await asyncio.sleep(agent.delay)

# =============================================================================
# EJECUCIÓN
# =============================================================================
async def main():
    # Si falla el puerto, reinicia kernel
    manager = ManagerAgent("manager@localhost", "password")
    await manager.start()
    try:
        while True:
            await asyncio.sleep(1)
    except KeyboardInterrupt:
        await manager.stop()

# await main()

✅ Archivo static/index.html actualizado.


In [ ]:
# Descomentar para ejecutar:
await main()

Agente manager@localhost iniciando...
🌐 GUI disponible en: http://localhost:10000/static/index.html
